In [ ]:
%load_ext autoreload
%autoreload 2

import os
import json

import sys
sys.path.append("../")

In [ ]:
sources = []
for entry in os.scandir("../data/stimuleringsregeling/"):
    if entry.is_file():
        with open(entry.path) as source_file:
            source = json.load(source_file)
        if any(doc for doc in source.get("documents", []) if doc.get("text", None)):
            sources.append(source)

In [ ]:
len(sources)